In [ ]:
# Activar Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
# Realiza el composite de q, u, y v y calcula el VIMF para niveles específicos.

import os

import pandas as pd
import numpy as np

import xarray as xr

# Datos básicos.

path = "/content/drive/MyDrive/Colab/Onset_Mexico/"
path_e = path + "data/Runmean/"
path_d = path + "data/"
path_r = path + "results/composite/"
path_v = path + "results/VIMF/"

# Si no existe la carpeta, la crea.
for i in [path_e, path_d, path_r, path_v]:
    if not os.path.exists(i): os.mkdir(i)

# Cantidad de días a promediar.
d = 20

# Variable, nivel de presión y región.
vars = [ "u", "v", "q" ]
vars_grib = [ "var" + x for x in np.arange(131, 134).astype(str) ]
levels = [ "1000", "975", "950", "925", "900", "875", "850", "825",
    "800", "775", "750", "700", "650", "600", "550", "500" ]

# Años comprendidos en CHIRPS.
years = list( range(1981, 2021) )

# Cargamos la información de desfase para cada año.
shift = []
delta = []
shift.append ( pd.read_csv(path_d + "shift_CHIRPS.csv",
    index_col = "Año", squeeze = True) )
shift.append( pd.read_csv(path_d + "shift_withdrawal_CHIRPS.csv",
    index_col = "Año", squeeze = True) )

# Fecha pivote del desfase. 
for i in [0, 1]:
    delta.append( shift[i].max() )
    shift[i] = delta[i] - shift[i]

onset = [ "onset", "withdrawal" ]

In [ ]:
# Realizamos el composite.

# Todas las variables.
for v in range(len(vars)):
    print(f"\nProcessing {vars[v]}:")

    # Todos los niveles.
    for n in range(len(levels)):
        fname = ( "onset_" + vars[v] + "_" + levels[n] + "_mean_20_dias.nc" )
        ds = xr.open_dataset( path_e + fname)
        ds = ds.rename( {vars_grib[v]: vars[v]} )
        # Onset y withdrawal, composite de la distribución espacial.
        for j in range(len(onset)):
            # Creamos un Dataset para cada año.
            ds_year = []
            for i in range( len(years) ):
                ds_year.append( ds.sel( time = ds.time.dt.year.isin(
                    years[i] ) ) )
                # Convertimos las fechas a dayofyear.
                ds_year[i] = ds_year[i].groupby(
                    "time.dayofyear" ).mean( "time" )
                # Coordenada para concatenar.
                ds_year[i].assign_coords( {"concat": i} )
                # Corregimos el dayofyear para los años bisiestos.
                if years[i] % 4 == 0: ds_year[i]["dayofyear"] = range(1, 366)
            # Alineamos las series de tiempo.
            for i in range( len(ds_year) ):
                ds_year[i] = ds_year[i].shift(
                    { "dayofyear": shift[j].iloc[i] } )
            # Concatenamos y promediamos por dayofyear.
            composite = xr.concat(ds_year, dim = "concat")
            composite = composite.mean("concat")
            if j == 1:
                # Agregamos los primeros días del
                # año después del último día del año.
                x = composite.isel(dayofyear = slice(0, 60))
                x["dayofyear"] = np.arange(366, 366 + 60)
                composite = xr.concat( (composite, x), dim = "dayofyear" )
            # Centramos el índice a la fecha de onset.
            composite = composite.sel( dayofyear = slice(
                delta[j] - 60, delta[j] + 60 ) )
            composite["dayofyear"] = range(-60, 61)
            # Establecemos un CRS.
            # Guardamos el netcdf.
            composite.to_netcdf( path_r + onset[j] + "_composite_" + vars[v]
                + "_" + levels[n] + "_mean_20_dias.nc" )
            
        print(f"{levels[n]},", end = " ")


Processing u:
1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 700, 650, 600, 550, 500, 
Processing v:
1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 700, 650, 600, 550, 500, 
Processing q:
1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 700, 650, 600, 550, 500, 

In [ ]:
# Unir viento.

# Todos los niveles.
for n in range(len(levels)):

    # Viento zonal y meridional.
    u = []
    for j in range(len(onset)):
        for v in [0, 1]:
            fname = ( onset[j] + "_composite_" + vars[v]
                + "_" + levels[n] + "_mean_20_dias.nc" )
            u.append( xr.open_dataset( path_r + fname ) )

        # Unimos en un solo netcdf.
        ds = xr.merge( ( u[0], u[1] ) )

        ds.to_netcdf( path_r + onset[j] + "_wind_"
            + levels[n] + "_mean_20_dias.nc" )
        
    print(f"{levels[n]},", end = " ")

1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 700, 650, 600, 550, 500, 

In [ ]:
# Concatenamos todos los niveles de presión.

ds = []
a = [64, 96]
b = [96, 128]
names = ["q", "wind"]

for i in range(len(names)):
    for j in range(len(onset)):
        files = os.listdir(path_r)[ (a[i] + j) : (b[i] + j) : 2]
        files = [path_r + x for x in files]
        ds = xr.open_mfdataset( files, combine = "nested",
            concat_dim = "plev", parallel = True )
        ds.to_netcdf( path_r + onset[j] + "_composite_"
            + names[i] + "_mean_20_dias.nc" )

In [ ]:
# Cálculo de VIMF.

for j in range(len(onset)):
    wind = xr.open_dataset( path_r + onset[j]
        + "_composite_wind_mean_20_dias.nc" )
    q    = xr.open_dataset( path_r + onset[j]
        + "_composite_q_mean_20_dias.nc"    )

    # Límites de cálculo.
    lim = (10e4, 7e4)

    # Preparamos las variables.
    q["u"] = q["q"]
    q = q.rename(q = "v")
    mf = q * wind

    # Divergencia.
    mf = mf.rename(u = "qu", v = "qv")
    mf["dqu_dx"] = ( ("dayofyear", "plev", "lat", "lon"),
        np.gradient( mf["qu"], axis = 2 ) )
    mf["dqv_dx"] = ( ("dayofyear", "plev", "lat", "lon"),
        np.gradient( mf["qv"], axis = 2 ) )

    #mf = mf.drop(["qu", "qv"])
    mf = mf.sel( plev = slice( int(10e4), int(7e4) ) )

    mf["vidmf"] = mf["dqu_dx"] + mf["dqv_dx"]
    #mf = mf.drop(["dqu_dx", "dqv_dx"])

    # Integral vertical.
    vidmf = mf.drop(["qu", "qv", "dqu_dx", "dqv_dx"]).mean("plev").copy()
    vimf = mf.drop(["dqu_dx", "dqv_dx", "vidmf"]).mean("plev").copy()

    vimf["qu"] = ( ("dayofyear", "lat", "lon"),
        np.trapz( mf["qu"], mf["plev"], axis = 1) )
    vimf["qv"] = ( ("dayofyear", "lat", "lon"),
        np.trapz( mf["qv"], mf["plev"], axis = 1) )
    vidmf["vidmf"] = ( ("dayofyear", "lat", "lon"),
        np.trapz( mf["vidmf"], mf["plev"], axis = 1) )

    vimf.to_netcdf(path_v +  onset[j] + "_composite_vimf_"
        + f"{lim[0]/100:.0f}" + "_" + f"{lim[1]/100:.0f}" + "_mean_20_dias.nc")
    vidmf.to_netcdf(path_v +  onset[j] + "_composite_vidmf_"
        + f"{lim[0]/100:.0f}" + "_" + f"{lim[1]/100:.0f}" + "_mean_20_dias.nc")